🌿# BOTANICAL DATA CLEANING & EDA

+ **Dataset:** pfaf_plants_merged.csv
+ **Focus:** Data Cleaning + Exploratory Analysis
+ **Goal Alignment:** Build the foundation plant database for the Botanical Decision Support Tool (BDST)

## 1. INTRODUCTION
### Project Goal

+ This notebook performs data cleaning and exploratory data analysis on a raw, messy botanical dataset containing plant properties, medicinal information, and cultivation details. 
+ This will form the foundation for later feature engineering, modeling, and the Botanical Decision Support Tool.

### Dataset Description

+ Rows: ~17,950
+ Columns: 27
+ Contains botanical names, common names, ratings, cultivation info, medicinal uses, etc.

#### Objectives

+  Load & inspect dataset
+  Identify data quality issues
+  Clean categorical + text data
+  Handle missing values
+  Standardize plant names
+  Remove/merge duplicates
+  Create clean features
+  Produce meaningful EDA insights
+  Save a cleaned dataset for Week 3

In [1]:
# 2. IMPORT LIBRARIES & LOAD DATA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

pd.set_option('display.max_colwidth', None)

In [2]:
# Load dataset
df = pd.read_csv("pfaf_plants_merged.csv")
df.head(2)

,use_keyword,latin_name_search,common_name_search,edibility_rating_search,medicinal_rating_search,plant_url,Care Requirements,Common Name,Common Names,Cultivation Details,...,Native Range,Other Uses,Other Uses Rating,Propagation,Range,Scientific Name,Special Uses,Summary,USDA hardiness,Weed Potential
0,Stomachic,Abelmoschus moschatus,"Musk Mallow,Musk Okra",2,3,https://pfaf.org/user/Plant.aspx?LatinName=Abelmoschus moschatus,Moist Soil; Half Hardy; Full sun,"Musk Mallow,Musk Okra","Musk Mallow,Musk Okra","Easily grown in a rich well-drained soil in a sunny position[238]. Tolerates a pH in the range 6 to 7.8. This species is not hardy in the colder areas of the country, it tolerates temperatures down to about -5°c and can be grown outdoors in the milder areas of the country[238]. The plant grows as a shrub in frost-free climates but is usually cut back to the ground in British winters. So long as these winters are not too cold, however, it can usually be grown as a herbaceous perennial with new shoots being produced freely from the root-stock. These flower in the summer[238]. It is probably wise to apply a good mulch to the roots in the autumn[K]. It is best to cut back the stems to about 15cm long in the spring even if they have not been killed back by the frost[238]. This will ensure an abundance of new growth and plenty of flowers in the summer. The musk mallow is widely cultivated in tropical climates for its many uses[238]. There is at least one named form, selected for its ornamental value. 'Mischief' is somewhat smaller than the species, reaching a height of 50cm[238].",...,"TEMPERATE ASIA: China (Hunan Sheng, Jiangxi Sheng, Guangdong Sheng, Guizhou Sheng, Yunnan Sheng, Guangxi Zhuangzu Zizhiqu), Taiwan TROPICAL ASIA: Bangladesh, India, Sri Lanka, Papua New Guinea, India (Andaman and Nicobar Islands), Cambodia, Laos, Thailand, Vietnam, Indonesia, Malaysia, Philippines AUSTRALASIA: Australia (Queensland, Western Australia (north), Northern Territory)","Essential Fibre Insecticide Oil Size An essential oil is obtained from the plant[238]. It is used as a food flavouring and in perfumery as a musk substitute[238]. However, it has been known to cause photosensitivity so this use has been largely discontinued[238]. An oil obtained from the seed contains 18.9% linoleic acid[240]. The oil is f high econmic value[266]. Total yields of oil are not given[K]. The seeds are used as an insecticide[4, 238]. Another report says that extracts of the fruits and upper parts of the plant show insecticidal activity[240]. A fibre is obtained from the stem bark[238]. It is used to make ropes[272]. A mucilage obtained from the roots is used as a size for paper[238].",3.0,"Seed - sow April in a greenhouse. The seed germinates best at a temperature around 24 - 24°c[238]. When large enough to handle, prick out the seedlings into individual pots of rich soil and plant them out after the last expected frosts[K]. The seed can also be sown in situ in late April in areas with warm summers. Cuttings of half-ripe wood, July in a frame[238].",S.E. Asia - Himalayas to China and Vietnam.,Abelmoschus moschatus - Medik.,Scented Plants,NaN,8-11,No
1,Stomachic,Abies grandis,"Grand Fir, Giant Fir, Lowland White Fir",2,2,https://pfaf.org/user/Plant.aspx?LatinName=Abies grandis,Semi-shade; Fully Hardy; Moist Soil; Full shade; Full sun,"Grand Fir, Giant Fir, Lowland White Fir","Grand Fir, Giant Fir, Lowland White Fir","Landscape Uses:Screen, Specimen. Prefers a good moist but not water-logged soil[1]. Grows well in heavy clay soils and succeeds in poor sandy soils[185]. Very shade tolerant, especially when young, but growth is slower in dense shade[81]. Intolerant of atmospheric pollution[1]. Trees succeeds in very exposed positions, even if the top is blown out by the wind the trees make one or more new tops and continue growing with no loss of vigour[11, 185]. Prefers slightly acid conditions down to a pH of about 5[200]. Prefers growing on a north-facing slope[

In [3]:
# 🔍 3. INITIAL DATA INSPECTION
# 3.1 Basic overview
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17950 entries, 0 to 17949
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   use_keyword              17950 non-null  object 
 1   latin_name_search        17950 non-null  object 
 2   common_name_search       16132 non-null  object 
 3   edibility_rating_search  17950 non-null  int64  
 4   medicinal_rating_search  17950 non-null  int64  
 5   plant_url                17950 non-null  object 
 6   Care Requirements        14283 non-null  object 
 7   Common Name              12700 non-null  object 
 8   Common Names             12700 non-null  object 
 9   Cultivation Details      14281 non-null  object 
 10  Edibility Rating         14283 non-null  float64
 11  Edible Uses              14283 non-null  object 
 12  Family                   14283 non-null  object 
 13  Image URLs               14283 non-null  object 
 14  Known Hazards         

,use_keyword,latin_name_search,common_name_search,edibility_rating_search,medicinal_rating_search,plant_url,Care Requirements,Common Name,Common Names,Cultivation Details,...,Native Range,Other Uses,Other Uses Rating,Propagation,Range,Scientific Name,Special Uses,Summary,USDA hardiness,Weed Potential
count,17950,17950,16132,17950.000000,17950.000000,17950,14283,12700,12700,14281,...,14256,14283,14283.000000,14266,14274,14283,8266,5920,14283,14283
unique,120,4131,3415,NaN,NaN,4131,739,3408,3408,3937,...,3382,2573,NaN,2075,2879,4130,183,1254,55,2
top,Diuretic,Lantana camara,"Sangre De Grado, Dragon's blood",NaN,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Lantana camara,Moist Soil; Full sun; Fully Hardy; Semi-shade,"Sangre De Grado, Dragon's blood","Sangre De Grado, Dragon's blood",Not known,...,Coming Soon,None known,NaN,Seed -,E. Asia - China.,Phytolacca dodecandra - L'Her.,Scented Plants,"Bloom Color: Yellow. Main Bloom Time: Early spring, Late spring, Mid spring. Form: Oval.",Coming soon,No
freq,1000,22,32,NaN,NaN,22,539,20,20,84,...,1771,3532,NaN,221,402,11,1237,49,2457,12005
mean,NaN,NaN,NaN,1.944011,2.353203,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.853112,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,1.440458,0.969669,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.447678,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,1.000000,2.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,2.000000,2.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,3.000000,3.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns.tolist()


['use_keyword',
 'latin_name_search',
 'common_name_search',
 'edibility_rating_search',
 'medicinal_rating_search',
 'plant_url',
 'Care Requirements',
 'Common Name',
 'Common Names',
 'Cultivation Details',
 'Edibility Rating',
 'Edible Uses',
 'Family',
 'Image URLs',
 'Known Hazards',
 'Medicinal Properties',
 'Medicinal Rating',
 'Native Range',
 'Other Uses',
 'Other Uses Rating',
 'Propagation',
 'Range',
 'Scientific Name',
 'Special Uses',
 'Summary',
 'USDA hardiness',
 'Weed Potential']

In [5]:
# 3.2 Missing values
df.isna().sum().sort_values(ascending=False)

Summary                    12030
Special Uses                9684
Common Name                 5250
Common Names                5250
Native Range                3694
Propagation                 3684
Range                       3676
Cultivation Details         3669
Image URLs                  3667
Medicinal Properties        3667
USDA hardiness              3667
Scientific Name             3667
Other Uses Rating           3667
Other Uses                  3667
Medicinal Rating            3667
Weed Potential              3667
Known Hazards               3667
Family                      3667
Edible Uses                 3667
Edibility Rating            3667
Care Requirements           3667
common_name_search          1818
latin_name_search              0
plant_url                      0
medicinal_rating_search        0
edibility_rating_search        0
use_keyword                    0
dtype: int64

In [6]:
# 3.3 Sample rows
df.sample(3)

,use_keyword,latin_name_search,common_name_search,edibility_rating_search,medicinal_rating_search,plant_url,Care Requirements,Common Name,Common Names,Cultivation Details,...,Native Range,Other Uses,Other Uses Rating,Propagation,Range,Scientific Name,Special Uses,Summary,USDA hardiness,Weed Potential
6893,Hepatic,Alocasia macrorrhizos,"Giant Taro, Giant Elephant Ear",3,2,https://pfaf.org/user/Plant.aspx?LatinName=Alocasia macrorrhizos,Moist Soil; Full shade; Semi-shade; Tender; Full sun,"Giant Taro, Giant Elephant Ear","Giant Taro, Giant Elephant Ear","Fodder: Bank Management: Standard Minor Global Crop A plant of the higher-rainfall areas of the lowland tropics, where it is cultivated at elevations up to 1,000 metres. It grows best in areas where annual daytime temperatures are within the range 20 - 25c, but can tolerate 10 - 32c[418 ].It prefers a mean annual rainfall in the range 2,500 - 3,500mm, but tolerates 2,000 - 4,200mm[418 ]. Grows best in a position in some shade[419 ]. Prefers a well-drained, humus-rich, fertile loam, though it is tolerant of a wide range of soil types[300 ]. Dislikes water-logged soils[300 ]. Prefers a pH in the range 5.7 - 6.3, tolerating 5 - 7.3[418 ]. Grows best in higher-rainfall areas of the lowland tropics[300 ]. Plants take from 400 - 600 days to mature, but the stems can then remain in a suitable condition for a considerable time[300 ]. Whilst many forms of this plant contain calcium oxalate crystals (see notes above on toxicity), cultivars have been developed in India that do not contain oxalates[300 ]. Like many species in the family Araceae, this plant has the ability to heat the flowering spadix as the pollen becomes ready for fertilization. This heat greatly increases the strength of the aroma released by the plant, thus attracting more pollinating insects. It can also have the effect of making the insects more active, thus increasing the level of fertilization[472 ]. Position in the garden: Border, Pots/Tubs, Shrubbery. They do well in pots or tubs but may only reach one to one and a half metres in height.",...,"TROPICAL ASIA: Solomon Islands, Indonesia (Kalimantan), Malaysia, Philippines AUSTRALASIA: Australia (Queensland)",Fibre Fodder Pollution Soil reclamation Agroforestry Uses: The plant grows rapidly in wetland conditions and has a propensity to accumulate metal contaminants such as zinc. It shows promise for use in sewerage treatment beds. Other Uses: A fibre is said to be obtained from the plant[454 ].,2.0,Seed - best sown as soon as it is ripe. Germinates best at 24c. Division of the rootstock as the plant is coming into growth. Off-sets will appear at the base as it matures which can be easily transplanted.,"E. Asia - Indian subcontinent, Malaysia.",Alocasia macrorrhizos - (L.) G.Don,Carbon Farming Food Forest,NaN,10-12,No
2961,Skin,Crassocephalum crepidioides,"Okinawa Spinach, Redflower Ragleaf, Fireweed",4,2,https://pfaf.org/user/Plant.aspx?LatinName=Crassocephalum crepidioides,Full sun; Full shade; Moist Soil; Semi-shade; Tender,"Okinawa Spinach, Redflower Ragleaf, Fireweed","Okinawa Spinach, Redflower Ragleaf, Fireweed","Grows well in soils that are rich in organic matter[ 299 ]. An easy to grow vegetable, especially suited to shady localities in home gardens and tree plantations[ 299 ]. Often cultivated as a food crop in the tropics, the plant has light, plumed seeds that are easily distributed by the wind. It has escaped from cultivation in many areas and become an invasive weed in some places[ 305 ].",...,"TEMPERATE ASIA: Yemen AFRICA: Ethiopia, Sudan, Kenya, Tanzania, Uganda, Burundi, Cameroon, Democratic Republic of the Congo, Gabon, Equatorial Guinea, Sao Tome and Principe, Côte D‘Ivoire, Ghana, Guinea, Guinea-Bissau, Liberia, Nigeria, Sierra Leone, Angola, Mozambique, Malawi, Zambia, Zimbabwe, Eswatini, South Africa (Eastern Cape, KwaZulu-Natal, Limpopo, Mpumalanga), Madagascar",Fodder Repellent Tannin Agroforestry Uses: Crassocephalum crepidioides has been used 

In [7]:
# check the column names
df.columns

Index(['use_keyword', 'latin_name_search', 'common_name_search',
       'edibility_rating_search', 'medicinal_rating_search', 'plant_url',
       'Care Requirements', 'Common Name', 'Common Names',
       'Cultivation Details', 'Edibility Rating', 'Edible Uses', 'Family',
       'Image URLs', 'Known Hazards', 'Medicinal Properties',
       'Medicinal Rating', 'Native Range', 'Other Uses', 'Other Uses Rating',
       'Propagation', 'Range', 'Scientific Name', 'Special Uses', 'Summary',
       'USDA hardiness', 'Weed Potential'],
      dtype='object')

In [8]:
#🧼 4. COLUMN CLEANUP & STRUCTURE FIXING
# 4.1 Standardize column names
df.columns = (
    df.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w_]", "", regex=True)
)
df.columns

Index(['use_keyword', 'latin_name_search', 'common_name_search',
       'edibility_rating_search', 'medicinal_rating_search', 'plant_url',
       'care_requirements', 'common_name', 'common_names',
       'cultivation_details', 'edibility_rating', 'edible_uses', 'family',
       'image_urls', 'known_hazards', 'medicinal_properties',
       'medicinal_rating', 'native_range', 'other_uses', 'other_uses_rating',
       'propagation', 'range', 'scientific_name', 'special_uses', 'summary',
       'usda_hardiness', 'weed_potential'],
      dtype='object')

In [9]:
# Find columns with similar base names
import itertools
from difflib import SequenceMatcher

def find_similar_column_names(columns, threshold=0.75):
    """
    Find pairs of column names that are textually similar.
    """
    similar_pairs = []

    for col1, col2 in itertools.combinations(columns, 2):
        similarity = SequenceMatcher(None, col1, col2).ratio()
        if similarity >= threshold:
            similar_pairs.append((col1, col2, round(similarity, 2)))

    return pd.DataFrame(similar_pairs, columns=["column_1", "column_2", "name_similarity"])


In [10]:
similar_name_cols = find_similar_column_names(df.columns)
similar_name_cols


,column_1,column_2,name_similarity
0,common_name_search,common_name,0.76
1,common_name_search,common_names,0.80
2,edibility_rating_search,medicinal_rating_search,0.83
3,edibility_rating_search,edibility_rating,0.82
4,medicinal_rating_search,medicinal_rating,0.82
5,common_name,common_names,0.96
6,edibility_rating,medicinal_rating,0.75


 + once duplicate columns have been found, these need to be inspected so that one can determine the next steps. 
 + to either drop or merge the columns

In [11]:
# Compare and inspect the duplicate columns
def compare_columns(df, col1, col2):
    """
    Compare two potentially redundant columns and return diagnostics.
    """

    result = {}

    # Missing values
    result["missing_col1"] = df[col1].isna().mean()
    result["missing_col2"] = df[col2].isna().mean()

    # Exact match rate (where both are non-null)
    both_non_null = df[[col1, col2]].dropna()
    if len(both_non_null) > 0:
        result["exact_match_rate"] = (both_non_null[col1] == both_non_null[col2]).mean()
    else:
        result["exact_match_rate"] = None

    # Numeric comparison
    if pd.api.types.is_numeric_dtype(df[col1]) and pd.api.types.is_numeric_dtype(df[col2]):
        result["type"] = "numeric"
        result["correlation"] = df[[col1, col2]].corr().iloc[0, 1]

    # Text comparison
    elif pd.api.types.is_string_dtype(df[col1]) and pd.api.types.is_string_dtype(df[col2]):
        result["type"] = "text"
        result["avg_len_col1"] = df[col1].dropna().str.len().mean()
        result["avg_len_col2"] = df[col2].dropna().str.len().mean()

    else:
        result["type"] = "mixed"

    # Recommendation logic
    if result["missing_col1"] < result["missing_col2"]:
        result["recommended_keep"] = col1
    else:
        result["recommended_keep"] = col2

    return pd.Series(result)


In [12]:
# A diagnostic Table to show how redundant the duplicate columns are
comparison_results = []

for _, row in similar_name_cols.iterrows():
    col1, col2 = row["column_1"], row["column_2"]
    comparison = compare_columns(df, col1, col2)
    comparison["column_1"] = col1
    comparison["column_2"] = col2
    comparison_results.append(comparison)

comparison_df = pd.DataFrame(comparison_results)
comparison_df



,missing_col1,missing_col2,exact_match_rate,type,recommended_keep,column_1,column_2,correlation
0,0.101281,0.292479,0.934724,mixed,common_name_search,common_name_search,common_name,NaN
1,0.101281,0.292479,0.934724,mixed,common_name_search,common_name_search,common_names,NaN
2,0.000000,0.000000,0.221448,numeric,medicinal_rating_search,edibility_rating_search,medicinal_rating_search,0.135656
3,0.000000,0.204290,0.998180,numeric,edibility_rating_search,edibility_rating_search,edibility_rating,0.998595
4,0.000000,0.204290,0.998180,numeric,medicinal_rating_search,medicinal_rating_search,medicinal_rating,0.997781
5,0.292479,0.292479,1.000000,mixed,common_names,common_name,common_names,NaN
6,0.204290,0.204290,0.220822,numeric,medicinal_rating,edibility_rating,medicinal_rating,0.132440


### Key take away from the Diagnostic Table above
+ From the diagnostic table, we see that there is a very weak match rate between medicinal rating and edibility rating, hence these two column represent two different aspect of the data. 
+ We do not drop or merge these columns, we keep them as they are. 
+ However the three common names columns, show strong correlation and they match exactly, hence these are duplicate columns, there is redundancy so we need to drop one of the columns
+ According to the diagnostics, the recommended keep is Common Names column. 
+ There is a strong correltion between medicinal_rating and medicinal_rating_search, and according to the diagnostic table we will keep medicinal_raating_search as it does not have missing values while the other has 20% missing values
+ There is also a strong correlation between edibility_rating_search and edibility_rating, we will keep the edibility_rating_search as it does not have missng values



In [13]:
def merge_text_columns(row, columns,
                       split_pattern=r'[;,/|]',
                       to_lower=True):
    values = []

    for col in columns:
        val = row.get(col)

        # Skip real NaN
        if pd.isna(val):
            continue

        val = str(val).strip()

        # Skip empty / fake missing values
        if val == '' or val.lower() in {'nan', 'none', 'null'}:
            continue

        parts = re.split(split_pattern, val)

        for p in parts:
            p = p.strip()
            if p:
                values.append(p.lower() if to_lower else p)

    if not values:
        return np.nan

    # Deduplicate while preserving order
    seen = set()
    unique_vals = []
    for v in values:
        if v not in seen:
            seen.add(v)
            unique_vals.append(v)

    return '; '.join(unique_vals)


In [14]:
df.columns.tolist()

['use_keyword',
 'latin_name_search',
 'common_name_search',
 'edibility_rating_search',
 'medicinal_rating_search',
 'plant_url',
 'care_requirements',
 'common_name',
 'common_names',
 'cultivation_details',
 'edibility_rating',
 'edible_uses',
 'family',
 'image_urls',
 'known_hazards',
 'medicinal_properties',
 'medicinal_rating',
 'native_range',
 'other_uses',
 'other_uses_rating',
 'propagation',
 'range',
 'scientific_name',
 'special_uses',
 'summary',
 'usda_hardiness',
 'weed_potential']

In [15]:
# Columns to merge
common_name_cols = [
    'common_name',
    'common_names',
    'common_name_search'
]


In [16]:
# Create merged column
df['common_names_merged'] = df.apply(
    merge_text_columns,
    axis=1,
    columns=common_name_cols
)


In [17]:
df['common_names_merged'].head(10)


0                                                                   musk mallow; musk okra
1                                                  grand fir; giant fir; lowland white fir
2                                                                            himalayan fir
3    china jute; velvetleaf; butterprint buttonweed jute; china mallow; indian velvet leaf
4                                                   sweet acacia; perfume acacia; huisache
5                                                                             musk milfoil
6                                                                                      NaN
7                                                                          siberian yarrow
8                                                                              basil thyme
9                                                                                      NaN
Name: common_names_merged, dtype: object

In [18]:
df = df.drop(columns=common_name_cols, errors='ignore')


In [19]:
df['common_names_merged'].describe()

count                               16132
unique                               3444
top       sangre de grado; dragon's blood
freq                                   32
Name: common_names_merged, dtype: object

In [20]:
# Define the duplicate columns to drop

columns_to_drop = [
    'edibility_rating',
    'medicinal_rating'
]



In [21]:
# Drop the duplicate columns 
df = df.drop(columns=columns_to_drop, errors='ignore')

In [22]:
# Check if the merged column is present
df.columns[df.columns.str.contains('common', case=False)]


Index(['common_names_merged'], dtype='object')

In [23]:
df.columns

Index(['use_keyword', 'latin_name_search', 'edibility_rating_search',
       'medicinal_rating_search', 'plant_url', 'care_requirements',
       'cultivation_details', 'edible_uses', 'family', 'image_urls',
       'known_hazards', 'medicinal_properties', 'native_range', 'other_uses',
       'other_uses_rating', 'propagation', 'range', 'scientific_name',
       'special_uses', 'summary', 'usda_hardiness', 'weed_potential',
       'common_names_merged'],
      dtype='object')

In [24]:
# Check the intergrity of the columns left 
df[['edibility_rating_search', 'medicinal_rating_search',]].describe()


,edibility_rating_search,medicinal_rating_search
count,17950.000000,17950.000000
mean,1.944011,2.353203
std,1.440458,0.969669
min,0.000000,0.000000
25%,1.000000,2.000000
50%,2.000000,2.000000
75%,3.000000,3.000000
max,5.000000,5.000000


### Column-by-column cleaning

In [25]:
## Rename column 
df["cultivation_region"] = df["range"]
df.columns

Index(['use_keyword', 'latin_name_search', 'edibility_rating_search',
       'medicinal_rating_search', 'plant_url', 'care_requirements',
       'cultivation_details', 'edible_uses', 'family', 'image_urls',
       'known_hazards', 'medicinal_properties', 'native_range', 'other_uses',
       'other_uses_rating', 'propagation', 'range', 'scientific_name',
       'special_uses', 'summary', 'usda_hardiness', 'weed_potential',
       'common_names_merged', 'cultivation_region'],
      dtype='object')

In [26]:
## Check similarity between the original range column and cultivation region

In [27]:
df["cultivation_region"].describe()

count                14274
unique                2879
top       E. Asia - China.
freq                   402
Name: cultivation_region, dtype: object

In [28]:
df["range"].describe()

count                14274
unique                2879
top       E. Asia - China.
freq                   402
Name: range, dtype: object

In [29]:
## once confirmed the same information exists, drop the old name column

In [30]:
df = df.drop(columns="range", errors='ignore')

In [31]:
## verify that the drop worked

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17950 entries, 0 to 17949
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   use_keyword              17950 non-null  object 
 1   latin_name_search        17950 non-null  object 
 2   edibility_rating_search  17950 non-null  int64  
 3   medicinal_rating_search  17950 non-null  int64  
 4   plant_url                17950 non-null  object 
 5   care_requirements        14283 non-null  object 
 6   cultivation_details      14281 non-null  object 
 7   edible_uses              14283 non-null  object 
 8   family                   14283 non-null  object 
 9   image_urls               14283 non-null  object 
 10  known_hazards            14283 non-null  object 
 11  medicinal_properties     14283 non-null  object 
 12  native_range             14256 non-null  object 
 13  other_uses               14283 non-null  object 
 14  other_uses_rating     

In [33]:
# Remove the references enclosed in the square brackets
df["cultivation_region"] = (
    df["cultivation_region"]
    .str.replace(r"\[[^\]]*\]", "", regex = True)
    .str.strip()
)

In [34]:
df["cultivation_region"].head()

0                                        S.E. Asia - Himalayas to China and Vietnam.
1    Western N. America - British Columbia to California, east to Montana and Idaho.
2                                     E. Asia - Himalayas from Afghanistan to Nepal.
3                 Asia - tropical. Naturalised in S.E. Europe and the Mediterranean.
4                 The original range is uncertain, but is probably tropical America.
Name: cultivation_region, dtype: object

In [35]:
df["cultivation_region"].str.contains(r"\[").sum()


0

In [36]:
## scale the code so that all columns with references enclosed be removed

In [37]:
# First find all object columns
object_cols = df.select_dtypes(include="object").columns.tolist()
object_cols

['use_keyword',
 'latin_name_search',
 'plant_url',
 'care_requirements',
 'cultivation_details',
 'edible_uses',
 'family',
 'image_urls',
 'known_hazards',
 'medicinal_properties',
 'native_range',
 'other_uses',
 'propagation',
 'scientific_name',
 'special_uses',
 'summary',
 'usda_hardiness',
 'weed_potential',
 'common_names_merged',
 'cultivation_region']

In [38]:
## once all object columns have been found,find the ones that have enclosed references that need to be removed.  
object_cols = df.select_dtypes(include="object").columns.tolist()

ref_cols = [
    c for c in object_cols
    if df[c].astype(str).str.contains(r"\[[^\]]*\]", na=False).any()
]

ref_cols


['cultivation_details',
 'edible_uses',
 'known_hazards',
 'native_range',
 'other_uses',
 'propagation',
 'summary']

In [39]:
## remove the references from the obj cols that have the references
for c in ref_cols:
    df[c] = (
        df[c]
        .astype(str)
        .str.replace(r"\[[^\]]*\]", "", regex=True)
        .str.strip()
        .replace({"": np.nan})
    )


In [40]:
## check if there are any references remaining in the columns
remaining = sum(
    df[c].astype(str).str.contains(r"\[[^\]]*\]", na=False).sum()
    for c in object_cols
)
remaining


0

In [41]:
## REDUNDANCY CHECK ON COLUMNS

In [42]:
# Care Requirements column vs Cultivation
[c for c in df.columns if "cult" in c.lower() or "care" in c.lower()]


['care_requirements', 'cultivation_details', 'cultivation_region']

In [43]:
col_a = "cultivation_details"
col_b = "care_requirements"


In [44]:
# check for missingness overlap between the two columns
a = df[col_a].fillna("").str.strip()
b = df[col_b].fillna("").str.strip()

mask_a = a.ne("")   # has content
mask_b = b.ne("")

stats = {
    "rows_total": len(df),
    f"{col_a}_filled": int(mask_a.sum()),
    f"{col_b}_filled": int(mask_b.sum()),
    "both_filled": int((mask_a & mask_b).sum()),
    f"only_{col_a}": int((mask_a & ~mask_b).sum()),
    f"only_{col_b}": int((~mask_a & mask_b).sum()),
    "both_empty": int((~mask_a & ~mask_b).sum()),
}
stats


{'rows_total': 17950,
 'cultivation_details_filled': 17950,
 'care_requirements_filled': 14283,
 'both_filled': 14283,
 'only_cultivation_details': 3667,
 'only_care_requirements': 0,
 'both_empty': 0}

In [45]:
len_a = a.str.len()
len_b = b.str.len()

len_a.describe(), len_b.describe()


(count    17950.000000
 mean       744.772591
 std        760.515954
 min          1.000000
 25%        156.000000
 50%        546.000000
 75%       1073.000000
 max       5303.000000
 Name: cultivation_details, dtype: float64,
 count    17950.000000
 mean        35.539443
 std         20.677121
 min          0.000000
 25%         28.000000
 50%         43.000000
 75%         51.000000
 max         86.000000
 Name: care_requirements, dtype: float64)

In [46]:
((len_a - len_b).describe())


count    17950.000000
mean       709.233148
std        750.759530
min        -38.000000
25%        113.000000
50%        502.000000
75%       1027.000000
max       5270.000000
dtype: float64

In [47]:
import re

def tokenize(s):
    return set(re.findall(r"[a-z]+", s.lower()))

# compute Jaccard only where both are present
pairs = df.loc[mask_a & mask_b, [col_a, col_b]].copy()

def jaccard(row):
    ta = tokenize(row[col_a])
    tb = tokenize(row[col_b])
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

pairs["jaccard"] = pairs.apply(jaccard, axis=1)
pairs["jaccard"].describe()


count    14283.000000
mean         0.039566
std          0.035676
min          0.000000
25%          0.016878
50%          0.030303
75%          0.051948
max          0.454545
Name: jaccard, dtype: float64

+ According to eye glance of the two columns the care requirements and culitvation details, it looked like they both contained the same
+ information although structured differently. 
+ however after doing the missningness overlap and the Jaccard similarity, it was discovered that care requirements is nested within the cultivation details
+ after perfoming the jaccard similarity( jaccard median is 0.03%), it was discovered that the content of the two columns is largely different and there is very low overlap
+ in conclusion, these two columns complement each other, hence we do not merge or drop the columns. 

In [48]:
# Compare other similar columns and check for similarity
# suspected similar columns that use the word "use"
[c for c in df.columns if "uses" in c.lower()]


['edible_uses', 'other_uses', 'other_uses_rating', 'special_uses']

In [49]:
use_cols = [
    "other_uses",
    "special_uses",
    "edible_uses",
    "uses"
]
use_cols = [c for c in use_cols if c in df.columns]


In [50]:
# check for missingness and coverage diagnostics
coverage = {
    col: df[col].fillna("").str.strip().ne("").sum()
    for col in use_cols
}

coverage


{'other_uses': 17950, 'special_uses': 8266, 'edible_uses': 17950}

+ According to the above coverage result, we have to inspect other_uses and edible_uses for content similarity as they are filled 100%
+ special_uses show that it is an independant column which might not have any content similarity with the other 2 columns
+ It is also necessary to do a content check between special_uses and other_uses aswell as with edible_uses

In [51]:
# Content Similarity check between the columns

def tokenize(s):
    return set(re.findall(r"[a-z]+", s.lower()))

import itertools

jaccard_results = []

for col_a, col_b in itertools.combinations(use_cols, 2):
    A = df[col_a].fillna("").str.strip()
    B = df[col_b].fillna("").str.strip()

    mask = A.ne("") & B.ne("")
    n_rows = int(mask.sum())

    if n_rows == 0:
        continue

    tmp = df.loc[mask, [col_a, col_b]].copy()

    def jaccard(row):
        ta = tokenize(row[col_a])
        tb = tokenize(row[col_b])
        if not ta or not tb:
            return 0.0
        return len(ta & tb) / len(ta | tb)

    sims = tmp.apply(jaccard, axis=1)

    jaccard_results.append({
        "col_a": col_a,
        "col_b": col_b,
        "rows_compared": n_rows,
        "median_jaccard": sims.median(),
        "mean_jaccard": sims.mean(),
        "max_jaccard": sims.max()
    })

jaccard_df = pd.DataFrame(jaccard_results)
jaccard_df


,col_a,col_b,rows_compared,median_jaccard,mean_jaccard,max_jaccard
0,other_uses,special_uses,8266,0.000000,0.021293,1.000000
1,other_uses,edible_uses,17950,0.119403,0.335741,1.000000
2,special_uses,edible_uses,8266,0.000000,0.002029,0.085714


+ According to the diagnostic ran, all columns have no content similarity hence we must keep all columns


#### Final Overal Missingness Analysis

In [52]:
#find missing values from each column and give the sum of those values
df.isna().sum().sort_values(ascending=False)

special_uses               9684
cultivation_region         3676
family                     3667
weed_potential             3667
usda_hardiness             3667
scientific_name            3667
other_uses_rating          3667
image_urls                 3667
medicinal_properties       3667
care_requirements          3667
common_names_merged        1818
edible_uses                   0
known_hazards                 0
latin_name_search             0
native_range                  0
other_uses                    0
cultivation_details           0
propagation                   0
plant_url                     0
summary                       0
medicinal_rating_search       0
edibility_rating_search       0
use_keyword                   0
dtype: int64

In [53]:
missing_pct = (
    df.isna()
      .mean()
      .sort_values(ascending=False)
      .to_frame(name="missing_fraction")
)

missing_pct["missing_percent"] = missing_pct["missing_fraction"] * 100
missing_pct.head(20)



,missing_fraction,missing_percent
special_uses,0.539499,53.949861
cultivation_region,0.204791,20.479109
family,0.204290,20.428969
weed_potential,0.204290,20.428969
usda_hardiness,0.204290,20.428969
scientific_name,0.204290,20.428969
other_uses_rating,0.204290,20.428969
image_urls,0.204290,20.428969
medicinal_properties,0.204290,20.428969
care_requirements,0.204290,20.428969


+ Upon running the missingness diagnostics, the summary column says it does not have any missing values, however from physical check of the column, it has several missing values
+ hence there is need to recheck and re-evaluate the stripping done earlier on text columns, as there may be empty strings that pandas is treating as having values 

In [54]:
# Diagnostic check on the summary column
df["summary"].head(10).apply(repr)



0                                                                                                                                                                                                                                          'nan'
1                                                                                                                                                                                                            'Form: Columnar, Upright or erect.'
2                                                                                                                                                                                                                                          'nan'
3                                                                                                                                                                                                                      'Form: Upright or erect.'
4    'Bloom Color: Yellow. Main Bloo

+ nan is treated as a string by pandas, hence there is need to convert to an actual NAN value

In [55]:
(df["summary"] == "").sum()


0

+ After this observation, there is need to convert all nan strings in all text columns to NAN values

In [56]:
text_cols = df.select_dtypes(include="object").columns


In [57]:
import numpy as np

df[text_cols] = (
    df[text_cols]
      .replace(
          to_replace=r"^(nan|none|null|unknown)?$",
          value=np.nan,
          regex=True
      )
)


In [62]:
# Once that is done, check for the missng values from the dataframe

In [58]:
missing_pct = (
    df.isna()
      .mean()
      .sort_values(ascending=False)
      .to_frame(name="missing_fraction")
)

missing_pct["missing_percent"] = missing_pct["missing_fraction"] * 100
missing_pct.head(15)


,missing_fraction,missing_percent
summary,0.670195,67.019499
special_uses,0.539499,53.949861
native_range,0.205794,20.579387
propagation,0.205237,20.523677
cultivation_region,0.204791,20.479109
cultivation_details,0.204401,20.440111
weed_potential,0.204290,20.428969
usda_hardiness,0.204290,20.428969
scientific_name,0.204290,20.428969
other_uses_rating,0.204290,20.428969


In [61]:
df["summary"].isna().sum()

12030

In [60]:

df["summary"].head(10)


0                                                                                                                                                                                                                                          NaN
1                                                                                                                                                                                                            Form: Columnar, Upright or erect.
2                                                                                                                                                                                                                                          NaN
3                                                                                                                                                                                                                      Form: Upright or erect.
4    Bloom Color: Yellow. Main Bloom Time: E

In [63]:
# after rerunning the summary missingness diagnostics, it now clearly shows that the summary column has a lot of missing values as expected after a physical check

In [64]:
# Final missingness check 
missing_pct = df.isna().mean().sort_values(ascending=False)
missing_pct.head(15)


summary                 0.670195
special_uses            0.539499
native_range            0.205794
propagation             0.205237
cultivation_region      0.204791
cultivation_details     0.204401
weed_potential          0.204290
usda_hardiness          0.204290
scientific_name         0.204290
other_uses_rating       0.204290
other_uses              0.204290
medicinal_properties    0.204290
known_hazards           0.204290
image_urls              0.204290
family                  0.204290
dtype: float64

In [66]:
# From the summary of the missing values, it is reaalised that many columns have the same missing fractions, which causes a question to arise and there is need to check why these column have exactly the same missing fraction.
# scientific name column should not have missing values as it the unique identifier, 

In [67]:
df["scientific_name"].isna().sum(), len(df)


(3667, 17950)

In [69]:
# drop the rows with no scientific names
df = df[df["scientific_name"].notna()].reset_index(drop=True)


In [70]:
# recheck missingness again 
missing_pct = df.isna().mean().sort_values(ascending=False)
missing_pct.head(15)


summary                0.585521
special_uses           0.421270
common_names_merged    0.110831
native_range           0.001890
propagation            0.001190
cultivation_region     0.000630
cultivation_details    0.000140
weed_potential         0.000000
usda_hardiness         0.000000
scientific_name        0.000000
other_uses_rating      0.000000
other_uses             0.000000
use_keyword            0.000000
latin_name_search      0.000000
known_hazards          0.000000
dtype: float64

In [71]:
# we conclude that most of the same missingness fractions were due to the missing scientific nmes, hence once we dropped those rows, the missingness dropped sharply

In [72]:
df.to_csv("pfaf_plants_clean_week2.csv", index=False)


### Botanical data cleaning Summary

+ Standardized schema and column names

+ Removed redundant columns

+ Diagnosed and retained complementary text fields

+ Normalized fake-missing values

+ Enforced scientific name as identifier

+ Characterized missingness without imputation